## Overview of ML in CV

**Machine Learning**: give computers the ability to make predictions or learn from data.

Supervised / Unsupervised. 강화학습 부분은 아직 CV쪽에서 application이 없으므로 언급하지 않음.

설명 자체는 그냥 머신러닝 쪽 입문설명.

현재 ML은 forefront of computer vision research. proved remarkable and useful.


### How is typically used?

* looking at raw pixel intensities or colors. 이미지 전체를 들여다보는 것. 언제나 most efficient way인 건 아니다.
* key point descriptors. : ORB 같은 애들.
* Histograms of image Gradient (HOGS) : very good and condensed descriptors of images.
* Pixel statistics를 보는 경우. dimensions, mean, variance, etc.

= 이미지에서 feature engineering하는 기본 원칙이기도 하다. 이렇게 빼낸 값들을 ML model에 넣는 것.

KNN 소개.


## HandWritten Digit Classifier.

숫자 이미지 받아서, 해당 숫자 인식하기. openCV에서 제공하는 digit file을 사용한다. (500 samples of numbers or digits btwn 0 and 9, 톤 5000개 이미지). 각각의 이미지는 20 * 20, 흑백이미지, 숫자는 흰색으로 표시됨.

numpy로 50 * 100 * 20 * 20 형태로 변형. 20 * 20은 flatten (1 * 400) 작업 진행. Train / Test로 7:3 비율 split.



**Flow 확인하기**
0. classifier 생성. 여기서는 KNN을 사용했지만, input으로 정해진 픽셀값을 받아서 결과를 리턴하는 함수면 된다.
1. grayscale the image
2. Gaussian Blur 
3. Canny Edge 추출 (blur와 같이, canny edge 추출은 find contours를 위한 전처리 작업으로 효과적)
4. Extract Contours (윤곽선 추출)
5. Get Bounding Rectangles
6. Filter Contours (tiny small contours that don't belong here. 즉 노이즈 제거)
7. Crop the image (bounding rectangle) + Blur more
8. Binarize image, and make into 20 * 20 square (inverse_threshold)
9. convert to 1 * 400 array
10. It finally becomes input into classifier.


내가 이해한 바로는

결국 '정해진 input 형식을 받아서 결과를 만들어내는 모델을 먼저 만든다', '예측할 실제 이미지 데이터에서 필요한 input만 잘라내어, 모델에 집어넣을 수 있는 input으로 변환한다', '모델에 input을 넣고 결과를 도출한다.'

In [5]:
import numpy as np
import cv2

# Let's take a look at our digits dataset
image = cv2.imread('./MasteringComputerVision-V1.03/Master OpenCV/images/digits.png')
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
small = cv2.pyrDown(image)
cv2.imshow('Digits Image', small)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Split the image to 5000 cells, each 20x20 size
# This gives us a 4-dim array: 50 x 100 x 20 x 20
cells = [np.hsplit(row,100) for row in np.vsplit(gray,50)]

# Convert the List data type to Numpy Array of shape (50,100,20,20)
x = np.array(cells)
print ("The shape of our cells array: " + str(x.shape))

# Split the full data set into two segments
# One will be used fro Training the model, the other as a test data set
train = x[:,:70].reshape(-1,400).astype(np.float32) # Size = (3500,400)
test = x[:,70:100].reshape(-1,400).astype(np.float32) # Size = (1500,400)

# Create labels for train and test data
k = [0,1,2,3,4,5,6,7,8,9]
train_labels = np.repeat(k,350)[:,np.newaxis]
test_labels = np.repeat(k,150)[:,np.newaxis]

# Initiate kNN, train the data, then test it with test data for k=3
knn = cv2.ml.KNearest_create()
knn.train(train, cv2.ml.ROW_SAMPLE, train_labels)
ret, result, neighbors, distance = knn.findNearest(test, k=3)

# Now we check the accuracy of classification
# For that, compare the result with test_labels and check which are wrong
matches = result == test_labels
correct = np.count_nonzero(matches)
accuracy = correct * (100.0 / result.size)
print("Accuracy is = %.2f" % accuracy + "%")


The shape of our cells array: (50, 100, 20, 20)
Accuracy is = 93.47%


### Defining some functions we will use to prepare an input image


In [11]:
import numpy as np
import cv2

# Define our functions

def x_cord_contour(contour):
    # This function take a contour from findContours
    # it then outputs the x centroid coordinates
    # 이미지의 윤곽선을 찾고, 그 이미지의 중심 x좌표를 반환하는 함수
#     if cv2.contourArea(contour) > 10:
    M = cv2.moments(contour)
    return (int(M['m10']/M['m00']))

def makeSquare(not_square):
    # This function takes an image and makes the dimenions square
    # It adds black pixels as the padding where needed
    # 이미지 받아서 square로 변환하는 함수. 필요하면 padding도 추가함.
    
    BLACK = [0,0,0]
    img_dim = not_square.shape
    height = img_dim[0]
    width = img_dim[1]
    #print("Height = ", height, "Width = ", width)
    if (height == width):
        square = not_square
        return square
    else:
        doublesize = cv2.resize(not_square,(2*width, 2*height), interpolation = cv2.INTER_CUBIC)
        height = height * 2
        width = width * 2
        #print("New Height = ", height, "New Width = ", width)
        if (height > width):
            pad = int((height - width)/2)
            #print("Padding = ", pad)
            doublesize_square = cv2.copyMakeBorder(doublesize,0,0,pad,\
                                                   pad,cv2.BORDER_CONSTANT,value=BLACK)
        else:
            pad = int((width - height)/2)
            #print("Padding = ", pad)
            doublesize_square = cv2.copyMakeBorder(doublesize,pad,pad,0,0,\
                                                   cv2.BORDER_CONSTANT,value=BLACK)
    doublesize_square_dim = doublesize_square.shape
    #print("Sq Height = ", doublesize_square_dim[0], "Sq Width = ", doublesize_square_dim[1])
    return doublesize_square


def resize_to_pixel(dimensions, image):
    # This function then re-sizes an image to the specificied dimenions
    
    buffer_pix = 4
    dimensions  = dimensions - buffer_pix
    squared = image
    r = float(dimensions) / squared.shape[1]
    dim = (dimensions, int(squared.shape[0] * r))
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    img_dim2 = resized.shape
    height_r = img_dim2[0]
    width_r = img_dim2[1]
    BLACK = [0,0,0]
    if (height_r > width_r):
        resized = cv2.copyMakeBorder(resized,0,0,0,1,cv2.BORDER_CONSTANT,value=BLACK)
    if (height_r < width_r):
        resized = cv2.copyMakeBorder(resized,1,0,0,0,cv2.BORDER_CONSTANT,value=BLACK)
    p = 2
    ReSizedImg = cv2.copyMakeBorder(resized,p,p,p,p,cv2.BORDER_CONSTANT,value=BLACK)
    img_dim = ReSizedImg.shape
    height = img_dim[0]
    width = img_dim[1]
    #print("Padded Height = ", height, "Width = ", width)
    return ReSizedImg

## Loading a new image, preprocessing it and classifying the digits

In [14]:
import numpy as np
import cv2

image = cv2.imread('./MasteringComputerVision-V1.03/Master OpenCV/images/numbers.jpg')
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
cv2.imshow("image", image)
cv2.imshow("gray", gray)
cv2.waitKey(0)

# Blur image then find edges using Canny 
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
cv2.imshow("blurred", blurred)
cv2.waitKey(0)

edged = cv2.Canny(blurred, 30, 150)
cv2.imshow("edged", edged)
cv2.waitKey(0)

# Find Contours
contours, _ = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#Sort out contours left to right by using their x cordinates
filtered_contours = [c for c in contours if cv2.contourArea(c) > 10]
contours = sorted(filtered_contours, key = x_cord_contour, reverse = False)

# Create empty array to store entire number
full_number = []

# loop over the contours
for c in contours:
    # compute the bounding box for the rectangle
    (x, y, w, h) = cv2.boundingRect(c)    
    
    #cv2.drawContours(image, contours, -1, (0,255,0), 3)
    #cv2.imshow("Contours", image)

    if w >= 5 and h >= 25:
        roi = blurred[y:y + h, x:x + w]
        ret, roi = cv2.threshold(roi, 127, 255,cv2.THRESH_BINARY_INV) # black background, white foreground
        
        squared = makeSquare(roi)
        
        final = resize_to_pixel(20, squared)
        
        cv2.imshow("final", final)
        final_array = final.reshape((1,400))
        final_array = final_array.astype(np.float32)
        
        ret, result, neighbours, dist = knn.findNearest(final_array, k=3)
        
        number = str(int(float(result[0])))
        full_number.append(number)
        
        # draw a rectangle around the digit, the show what the
        # digit was classified as
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)
        cv2.putText(image, number, (x , y + 155),
            cv2.FONT_HERSHEY_COMPLEX, 2, (255, 0, 0), 2)
        cv2.imshow("image", image)
        cv2.waitKey(0) 
        
cv2.destroyAllWindows()
print ("The number is: " + ''.join(full_number))

The number is: 12540
